# Notebook Template
template for jupyter notebook
- <a href='#1'>1. task1</a> 
- <a href='#2'>2. task2</a> 
- <a href='#3'>3. task3</a>

In [1]:
import sys
import os 

import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

sys.path.append('../')
import conf

ModuleNotFoundError: No module named 'conf'

In [2]:
# global settings
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns',1000)
pd.set_option('display.width',100)
sns.set(rc={'figure.figsize':(11,4)})

NameError: name 'conf' is not defined

In [3]:
# global variables
DEFAULT_MISSING_VALUE = 0
FONT = fm.FontProperties(fname = os.path.join(conf.lib_dir,'simsun.ttc'))

In [4]:
# functions
def __dummy():
    pass